<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/Violence_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip uninstall emoji
!pip install emoji==1.7.0
!pip install text2emotion

import text2emotion as te
from IPython.display import clear_output 

clear_output()

#Datasets

[link 1](https://www.kaggle.com/datasets/adiamaan/movie-subtitle-dataset)
[link 2](https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset)
[link 3](https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git)
[link 4](https://www.kaggle.com/datasets/ashishpatel26/facial-expression-recognitionferchallenge)

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
!kaggle datasets download -d adiamaan/movie-subtitle-dataset
!kaggle datasets download -d ashishpatel26/facial-expression-recognitionferchallenge
!unzip \*.zip && rm *.zip
!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml

clear_output()

In [3]:
os.makedirs('/content/NonViolence/', exist_ok=True)
os.makedirs('/content/Violence/', exist_ok=True)

nv = '/content/Real Life Violence Dataset/NonViolence/'
v = '/content/Real Life Violence Dataset/Violence/'

for V in [v, nv]:
  for i in os.listdir(V):
    R = V+i
    os.replace(R, R.replace('Real Life Violence Dataset/', ''))

In [4]:
path = '/content/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/'
choices = ['/content/NonViolence/', '/content/NonViolence/', '/content/Violence/', '/content/Violence/']

for id, V in enumerate(['non-violent/cam1/', 'non-violent/cam2/', 'violent/cam1/', 'violent/cam2/']):
  for i in os.listdir(path+V):
    os.replace(path+V+i, choices[id]+i)

In [5]:
os.makedirs('/content/data/', exist_ok=True)
os.replace('/content/NonViolence/', '/content/data/NonViolence/')
os.replace('/content/Violence/', '/content/data/Violence/')

In [6]:
subs = pd.read_csv('/content/movies_subtitles.csv')

In [7]:
subs

,start_time,end_time,text,imdb_id
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709
1,61.687,63.354,Don't anybody move!,tt0114709
2,64.398,66.482,"Now, empty that safe!",tt0114709
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709
...,...,...,...,...
10358491,6309.266,6310.260,60 Euro.,tt1576450
10358492,6326.766,6330.213,The slippers are sold out!\nWhat do we make now?,tt1576450
10358493,6331.850,6333.046,Let's see.,tt1576450
10358494,6335.225,6337.547,One day after the other.,tt1576450


#Text

In [8]:
sample = "I was asked to sign a third party contract a week out from stay"
te.get_emotion(sample)

{'Happy': 0.0, 'Angry': 0.25, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 0.75}

In [9]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def convert_abb(x):
    if type(x) != str:
      return str(x)
    word_list = x.split()
    r_string = []
    for word in word_list:
        r_string.append(abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word)
    return ' '.join(r_string)

In [10]:
subs['text'] = subs.text.apply(convert_abb)

In [ ]:
# ss = subs.text.apply(lambda x: te.get_emotion(x))

for id, sample in enumerate(subs.text[:2000000]):
  if id%100==0:
    print(id)
  emotions = te.get_emotion(sample)
  for pred in emotions:
    with open(f"/content/drive/MyDrive/University/Violence_Detection/Text_Emotion.txt", 'a+') as f:
      f.write(pred+',')

13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300


#Video

In [ ]:
# import math
# import numpy as np
# import pandas as pd

# import cv2
# import seaborn as sns
# from matplotlib import pyplot

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report

# import tensorflow as tf
# from tensorflow.keras import optimizers
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Flatten, Dense, Conv2D, GlobalAveragePooling2D
# from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
# from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from keras.utils import np_utils


df = pd.read_csv('/content/fer2013/fer2013/fer2013.csv')
print(df.shape)
df.head()

(35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
df.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
df.emotion.value_counts()

3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48).astype('float32'))
img_array = np.stack(img_array, axis = 0)
img_array.shape

img_features = []

for i in range(len(img_array)):
    temp = cv2.cvtColor(img_array[i], cv2.COLOR_GRAY2RGB)
    img_features.append(temp)

img_features = np.array(img_features)
print(img_features.shape)

(35887, 48, 48, 3)


In [ ]:
le = LabelEncoder()

img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)img_labels.shape

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}


(35887, 7)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_features, 
                                                      img_labels, 
                                                      shuffle = True, 
                                                      stratify = img_labels, 
                                                      test_size = 0.1, 
                                                      random_state = 42)

img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((32298, 48, 48, 3), (3589, 48, 48, 3), (32298, 7), (3589, 7))

In [ ]:
import math
import numpy as np
import pandas as pd

import cv2
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils



X_train = X_train / 255.
X_valid = X_valid / 255.

vgg = tf.keras.applications.VGG19(weights = 'imagenet',
                                  include_top = False,
                                  input_shape = (48, 48, 3))

def build_model(bottom_model, classes):
    model = bottom_model.layers[-2].output
    model = GlobalAveragePooling2D()(model)
    model = Dense(classes, activation = 'softmax', name = 'out_layer')(model)
    
    return model
head = build_model(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = head)

model.summary()

80134624/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                              

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', 
                               min_delta = 0.00005, 
                               patience = 11,
                               verbose = 1, 
                               restore_best_weights = True,)

lr_scheduler = ReduceLROnPlateau(monitor = 'val_accuracy', 
                                 factor = 0.5, 
                                 patience = 7,
                                 min_lr = 1e-7,
                                 verbose = 1,)

callbacks = [early_stopping,lr_scheduler,]
train_datagen = ImageDataGenerator(rotation_range = 15,
                                   width_shift_range = 0.15,
                                   height_shift_range = 0.15,
                                   shear_range = 0.15,
                                   zoom_range = 0.15,
                                   horizontal_flip = True,)
train_datagen.fit(X_train)
# batch size of 32 performs the best.
batch_size = 32 
epochs = 25
optims = [optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),]

model.compile(loss = 'categorical_crossentropy',
              optimizer = optims[0],
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_datagen.flow(X_train, 
                                       y_train, 
                                       batch_size = batch_size),
                                       validation_data = (X_valid, y_valid),
                                       steps_per_epoch = len(X_train) / batch_size,
                                       epochs = epochs,
                                       callbacks = callbacks,
                                       use_multiprocessing = True)

Epoch 1/25
1009/1009 [==============================] - 80s 61ms/step - loss: 1.5546 - accuracy: 0.3806 - val_loss: 1.2992 - val_accuracy: 0.5102 - lr: 1.0000e-04
Epoch 2/25
1009/1009 [==============================] - 62s 61ms/step - loss: 1.2815 - accuracy: 0.5093 - val_loss: 1.1846 - val_accuracy: 0.5536 - lr: 1.0000e-04
Epoch 3/25
1009/1009 [==============================] - 63s 62ms/step - loss: 1.1876 - accuracy: 0.5463 - val_loss: 1.1014 - val_accuracy: 0.5848 - lr: 1.0000e-04
Epoch 4/25
1009/1009 [==============================] - 62s 61ms/step - loss: 1.1254 - accuracy: 0.5739 - val_loss: 1.0630 - val_accuracy: 0.5899 - lr: 1.0000e-04
Epoch 5/25
1009/1009 [==============================] - 62s 61ms/step - loss: 1.0778 - accuracy: 0.5944 - val_loss: 1.0153 - val_accuracy: 0.6160 - lr: 1.0000e-04
Epoch 6/25
1009/1009 [==============================] - 71s 70ms/step - loss: 1.0387 - accuracy: 0.6120 - val_loss: 1.0332 - val_accuracy: 0.6247 - lr: 1.0000e-04
Epoch 7/25
1009/1009 [

In [ ]:
mapper = {
    0: 'anger',
    1: 'disgust',
    2: 'fear',
    3: 'happiness',
    4: 'sadness',
    5: 'surprise',
    6: 'neutral'
}

model.load_weights("/content/drive/MyDrive/University/Violence_Detection/model.h5")

In [ ]:
model_yaml = model.to_json()
with open("/content/drive/MyDrive/University/Violence_Detection/model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
model.save("/content/drive/MyDrive/University/Violence_Detection/model.h5")

In [ ]:
category = 'NonViolence'
path = f'/content/data/{category}/'
for id, file_p in enumerate(os.listdir(path)):
  video_file = path + file_p
  cap = cv2.VideoCapture(video_file)
  print('The Code is Running On Video: ', id)
  while(cap.isOpened()):
      flag, frame = cap.read()
      if flag == False:
          break
      image=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
      faces = face_cascade.detectMultiScale(image, 1.3, 5)
      for (x, y, w, h) in faces:
          face_section = image[y:y+h, x:x+w]
          face_section = cv2.resize(face_section, (48, 48))
          face_section = cv2.cvtColor(face_section, cv2.COLOR_GRAY2RGB)
          pred = mapper[np.argmax(model.predict(face_section.reshape(1,48,48,3))[0])]
          with open(f"/content/drive/MyDrive/University/Violence_Detection/{category}.txt", 'a+') as f:
            f.write(pred+',')
  
  cap.release()

#Test Part

In [ ]:
def get_emotions(path):
  with open(path, 'r') as f:
    data = f.read()  
  data = data.replace('anger', 'Angry')
  data = data.replace('disgust', 'Angry')
  data = data.replace('fear', 'Fear')
  data = data.replace('happiness', 'Happy')
  data = data.replace('sadness', 'Sad')
  data = data.replace('surprise', 'Surprise')
  data = data.replace('neutral', 'Happy')
  data = data.split(',')
  data.remove('')
  return np.array(data)

In [ ]:
path = '/content/drive/MyDrive/University/Violence_Detection/NonViolence.txt'
get_emotions(path)

In [ ]:
with open('/content/drive/MyDrive/University/Violence_Detection/NonViolence.txt', 'r') as f:
  data = f.read()

In [ ]:
data = data.replace('anger', 'Angry')
data = data.replace('disgust', 'Angry')
data = data.replace('fear', 'Fear')
data = data.replace('happiness', 'Happy')
data = data.replace('sadness', 'Sad')
data = data.replace('surprise', 'Surprise')
data = data.replace('neutral', 'Happy')


# data = data.replace('anger', 'Angry').replace('disgust', 'Angry').replace('fear', 'Fear').replace('happiness', 'Happy').replace('sadness', 'Sad').replace('surprise', 'Surprise').replace('neutral', 'Happy')

In [ ]:
data = data.split(',')
data.remove('')
data = np.array(data)
np.unique(data), len(data)

(array(['Angry', 'Fear', 'Happy', 'Sad', 'Surprise'], dtype='<U8'), 20891)

In [ ]:
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M    

In [ ]:
Category_names = {0: 'Happy', 
            1: 'Angry',
            2: 'Surprise', 
            3: 'Sad',
            4: 'Fear'}

Category = {'Happy':0, 'Angry':1 , 'Surprise':2, 'Sad':3, 'Fear': 4}

T = [Category[c] for c in data]

transition_mt = transition_matrix(T)

print('         ', end='')
print(' '.join('{0}'.format(x) for x in Category.keys()))
i=0
for row in transition_mt:
   print((list(Category.keys())[i]).rjust(8, ' '), end=' ')
   print('  '.join('{0:.2f}'.format(x) for x in row))
   i += 1


         Happy Angry Surprise Sad Fear
   Happy 0.82  0.03  0.11  0.04  0.01
   Angry 0.28  0.49  0.18  0.03  0.02
Surprise 0.29  0.05  0.63  0.02  0.00
     Sad 0.42  0.03  0.07  0.47  0.01
    Fear 0.46  0.08  0.13  0.13  0.20


In [ ]:
start_state=0
pre_state = start_state
print(Category_names[start_state], '--->', end=' ')

for i in range(10):
  current_state = np.random.choice(list(Category.values()), p=transition_mt[pre_state])
  print(Category_names[current_state], '--->', end=' ')
  pre_state = current_state
print('End')

Happy ---> Sad ---> Sad ---> Happy ---> Happy ---> Happy ---> Happy ---> Happy ---> Happy ---> Happy ---> Surprise ---> End


In [ ]:
def find_prob(seq, A, pi):
    start_state = seq[0]
    prob = pi[start_state]
    prev_state, curr_state = start_state, start_state
    for i in range(1, len(seq)):
        curr_state = seq[i]
        prob *= A[prev_state][curr_state]
        prev_state = curr_state
    return prob

import scipy.linalg
values, left = scipy.linalg.eig(transition_mt, right = False, left = True)
pi = left[:,0]
pi_normalized = [(x/np.sum(pi)).real for x in pi]


print(find_prob([1, 2, 2, 0], transition_mt, pi_normalized))

0.002238818793396924
